In [50]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.agents import tool
from langchain.document_loaders import TextLoader, UnstructuredPDFLoader, YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
import textwrap
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.tools import Tool
import langchain
import json
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain.schema import Document
from typing import Iterator, List, Literal, Optional, Sequence, Union
from langchain.document_loaders.blob_loaders import FileSystemBlobLoader
import os
import openai
from langchain.document_loaders import TextLoader


In [51]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [52]:


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']


##  Download Video in Save directory
## Transcribe the videos to text
## keep the transcript in document 

In [53]:

#  Yoga lecture videos
urls = ["https://www.youtube.com/watch?v=_xASV0YmROc"]

# Directory to save audio files
save_dir = "/Users/bpakra200/Downloads/YouTubeYoga3"

try:
    # Transcribe the videos to text
    loader = GenericLoader(YoutubeAudioLoader(urls, save_dir), OpenAIWhisperParser())

    docs = loader.load()
    print("After load")
    if len(docs) == 0:
        print("No documents loaded.")
    else:
        for doc in docs:
            # Process the loaded documents
            print(doc)
except Exception as e:
    print("Error loading documents:", str(e))    
 

[youtube] Extracting URL: https://www.youtube.com/watch?v=_xASV0YmROc
[youtube] _xASV0YmROc: Downloading webpage
[youtube] _xASV0YmROc: Downloading ios player API JSON
[youtube] _xASV0YmROc: Downloading android player API JSON
[youtube] _xASV0YmROc: Downloading m3u8 information
[info] _xASV0YmROc: Downloading 1 format(s): 140
[download] /Users/bpakra200/Downloads/YouTubeYoga3/The 3-Minute Yoga Routine To Jump Start Your Mornings.m4a has already been downloaded
[download] 100% of    2.70MiB


ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


Error loading documents: ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


In [54]:
# Returns a list of Documents, which can be easily viewed or parsed
len(docs)

1

In [55]:
# Combine doc
combined_docs = [doc.page_content for doc in docs]
text = " ".join(combined_docs)

Although HuggingFaceEmbeddings is a free and user-friendly option to get started with embeddings, OpenAI also offers an API for text embeddings5, which might perform much better. With LangChain, it is straightforward to use the OpenAI API for embeddings:

In [56]:
embeddings = OpenAIEmbeddings()

# Storing Embeddings
Note that Chroma doesn't automatically persist the embeddings, so you'll need to provide an additional parameter persist_directory to ensure they're saved along with the documents:

In [57]:
PERSIST_DIRECTORY_OPENAI = "../tubegptdb"
 
tubegptdb = Chroma.from_documents(
    documents=docs, embedding= embeddings, persist_directory=PERSIST_DIRECTORY_OPENAI
)
tubegptdb.similarity_search_with_score("What was the yoga move?", k=2)

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


[(Document(page_content='there is a stuffed santa claus doll on a wreath on the back of a truck\nthere is a stuffed santa claus doll on a wreath on the back of a car\nthere is a stuffed santa claus on a wreath on the back of a car\nthere is a stuffed santa claus sitting on a wreath on the back of a car\nthere is a stuffed santa claus sitting on a wreath on the back of a golf cart\nthere is a stuffed santa claus on a wreath on the back of a golf cart\nthere is a stuffed santa claus sitting on a wreath on the back of a truck\nthere is a stuffed animal that is sitting on a wreath\nthere is a wreath with a santa clause on it on the back of a car\nthere is a stuffed animal in a wreath on the back of a car\nthere is a stuffed santa claus on a wreath on the back of a truck\nthere is a stuffed animal in a wreath on the back of a truck\nthere is a stuffed santa claus sitting on a christmas wreath\nthere is a stuffed santa claus doll sitting on a christmas wreath\nthere is a stuffed animal that 

# Loading Embeddings
Loading the embeddings is just as easy, you just need to provide the persist_directory and embedding_function parameters:

In [58]:
tubegptdb.persist()

# save openai embedding

In [59]:
PERSIST_DIRECTORY_OPENAI = "../tubegptdb"
embeddings = OpenAIEmbeddings()
tubegptdb = Chroma(persist_directory=PERSIST_DIRECTORY_OPENAI, embedding_function=embeddings)
tubegptdb.similarity_search_with_score("it is yoga", k=2)

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


[(Document(page_content='there is a stuffed santa claus doll on a wreath on the back of a truck\nthere is a stuffed santa claus doll on a wreath on the back of a car\nthere is a stuffed santa claus on a wreath on the back of a car\nthere is a stuffed santa claus sitting on a wreath on the back of a car\nthere is a stuffed santa claus sitting on a wreath on the back of a golf cart\nthere is a stuffed santa claus on a wreath on the back of a golf cart\nthere is a stuffed santa claus sitting on a wreath on the back of a truck\nthere is a stuffed animal that is sitting on a wreath\nthere is a wreath with a santa clause on it on the back of a car\nthere is a stuffed animal in a wreath on the back of a car\nthere is a stuffed santa claus on a wreath on the back of a truck\nthere is a stuffed animal in a wreath on the back of a truck\nthere is a stuffed santa claus sitting on a christmas wreath\nthere is a stuffed santa claus doll sitting on a christmas wreath\nthere is a stuffed animal that 

##  Prompt Engineering Template

In [60]:
template = """You're a Professor who helps students on Youtube contents .
 
{context}
 
Answer with Accurate data   to the question and the way Professor speaks and only depends on provided Data. 
 
Question: {question}
Answer:"""
 
prompt = PromptTemplate(template=template, input_variables=["context", "question"])
print(
    prompt.format(
        context="A youtube user asking about contents ",
        question="Why do we need to zero out the gradient before backprop at each step?",
    )
)

You're a Professor who helps students on Youtube contents .
 
A youtube user asking about contents 
 
Answer with Accurate data   to the question and the way Professor speaks and only depends on provided Data. 
 
Question: Why do we need to zero out the gradient before backprop at each step?
Answer:


## implement Retriver and Reader with Vector database

In [61]:
chain_type_kwargs = {"prompt": prompt}
chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=tubegptdb.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
)

In [62]:
query = "What was the move after heading down like a Cat"
response = chain.run(query)
print_response(response)

After heading down like a cat, the move was to continue playing frisbee in the grass on the golf
course.


In [63]:
query = "When I should focus on my core"
response = chain.run(query)
print_response(response)

When you should focus on your core depends on your specific goals and needs. If you are looking to
improve your overall strength and stability, it is important to incorporate core exercises into your
regular workout routine. This can include exercises such as planks, Russian twists, and bicycle
crunches. Additionally, if you participate in activities that require a strong core, such as sports
or weightlifting, it is important to prioritize core training to enhance your performance and reduce
the risk of injury. It is recommended to consult with a fitness professional or personal trainer to
develop a personalized plan that suits your individual needs and goals.


In [64]:
query = "When I should focus on my core"
response = chain.run(query)
print_response(response)

When you should focus on your core depends on your specific goals and needs. If you are looking to
improve your overall strength and stability, it is important to incorporate core exercises into your
regular workout routine. This can include exercises such as planks, Russian twists, and bicycle
crunches. Additionally, if you participate in activities that require a strong core, such as sports
or weightlifting, it is important to prioritize core training to enhance your performance and reduce
the risk of injury. It is recommended to consult with a fitness professional or personal trainer to
develop a personalized plan that suits your individual needs and goals.


In [65]:
import torch
import torchvision.transforms as transforms
import cv2
from PIL import Image
import pafy
from transformers import BlipProcessor, BlipForConditionalGeneration
import youtube_dl

## Computer Vision Model with BLIP2 

In [66]:
import torch
import torchvision.transforms as transforms
import cv2
from PIL import Image
import pafy
from transformers import BlipProcessor, BlipForConditionalGeneration,AutoProcessor
import youtube_dl

In [67]:
import inspect

if not hasattr(inspect, 'getargspec'):
    inspect.getargspec = inspect.getfullargspec

In [68]:
video_url = 'https://www.youtube.com/watch?v=6n9ESFJTnHs'

#video_url = 'https://www.youtube.com/watch?v=orbkg5JH9C8'

# Download the video using youtube_dl and extract the frames using OpenCV
ydl_opts = {
    'format': 'best',
    'outtmpl': 'video.mp4'
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

[youtube] 6n9ESFJTnHs: Downloading webpage
[download] video.mp4 has already been downloaded
[download] 100% of 26.85MiB


## find Framerate of the video

In [69]:

# Create a VideoCapture object
cap = cv2.VideoCapture("demo1.mp4")

# Get the frame rate property
frame_rate = cap.get(cv2.CAP_PROP_FPS)

# Print the frame rate
print("The frame rate is {} frames per second".format(frame_rate))

# Release the VideoCapture object
cap.release() 

The frame rate is 29.97002997002997 frames per second


## set lower framerate 

In [70]:
# Create a VideoCapture object
cap = cv2.VideoCapture("demo1.mp4")

# Set the desired frame rate to 20
cap.set(cv2.CAP_PROP_FPS, 10)
cap.release() 

In [71]:
# Create a VideoCapture object
cap = cv2.VideoCapture("video.mp4")

# Set the desired frame rate to 20
cap.set(cv2.CAP_PROP_FPS, 1)
cap.release() 

## set model Different Models for testing

In [95]:
#model_id = "Salesforce/blip-image-captioning-large"


model_id = "Salesforce/blip-image-captioning-large"

#model_id = "Salesforce/blip2-opt-2.7b"

# Set up the Hugging Face model 
model = BlipForConditionalGeneration.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)

## Generate text from Image

In [97]:
import torch

def generate_text(image, prompt, model, processor, device):
    inputs = processor(image, text=prompt, return_tensors="pt")
    out = model.generate(**inputs)
    generated_text = processor.decode(out[0], skip_special_tokens=True)
    return generated_text

## Create Captions from image by base models

In [73]:
def caption_youtube(path):

                # Open the video file
            cap = cv2.VideoCapture(path)
            cap.set(cv2.CAP_PROP_FPS, 10)
            # Loop through the frames of the video
            combined_docs = []
  
            while cap.isOpened():
                        # Read the next frame
                        ret, frame = cap.read()
                    
                        # If we've reached the end of the video, break out of the loop
                        if not ret:
                            break
                    
                        # Convert the frame to a PIL image and apply the image transformation pipeline
                        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                        # image = image_transforms(image)
                    
                        # # Add batch dimension to the image tensor
                        # image = image.unsqueeze(0)
                        inputs = processor(image, return_tensors="pt")
                        out = model.generate(**inputs)
                        caption = processor.decode(out[0], skip_special_tokens=True)
                        # Print the caption for the frame

                        print(caption)
                        if caption not in combined_docs:
                             combined_docs.append(caption)
                      
                    # Release the video file
                    
            cap.release()
            return combined_docs

## device 

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

## Create Caption with Advance Model like Blip2

In [98]:
def caption_youtube_blip2(path):

                # Open the video file
            cap = cv2.VideoCapture(path)
            cap.set(cv2.CAP_PROP_FPS, 10)
            # Loop through the frames of the video
            combined_docs = []
            prompt = " Notice All Human Activity carefully and Notice All numbers and words in images"
            while cap.isOpened():
                        # Read the next frame
                        ret, frame = cap.read()
                    
                        # If we've reached the end of the video, break out of the loop
                        if not ret:
                            break
                    
                        # Convert the frame to a PIL image and apply the image transformation pipeline
                        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                        # image = image_transforms(image)
                        caption = generate_text(image, prompt, model, processor, device)
                        print(caption)
                        if caption not in combined_docs:
                             combined_docs.append(caption)
                      
                    # Release the video file
                    
            cap.release()
            return combined_docs

In [100]:

combined_docs = caption_youtube("/Users/bpakra200/tubegpt/tubegpt/src/notebook/demo1.mp4")

there is a stuffed santa claus doll on a wreath on the back of a truck
there is a stuffed santa claus doll on a wreath on the back of a car
there is a stuffed santa claus on a wreath on the back of a car
there is a stuffed santa claus sitting on a wreath on the back of a car
there is a stuffed santa claus sitting on a wreath on the back of a golf cart
there is a stuffed santa claus on a wreath on the back of a golf cart
there is a stuffed santa claus sitting on a wreath on the back of a truck
there is a stuffed santa claus on a wreath on the back of a car
there is a stuffed santa claus sitting on a wreath on the back of a truck
there is a stuffed santa claus sitting on a wreath on the back of a truck
there is a stuffed santa claus on a wreath on the back of a car
there is a stuffed animal that is sitting on a wreath
there is a wreath with a santa clause on it on the back of a car
there is a stuffed animal in a wreath on the back of a car
there is a stuffed santa claus on a wreath on th

In [ ]:
combined_docs = caption_youtube_blip2("/Users/bpakra200/tubegpt/tubegpt/src/notebook/demo1.mp4")

## Save Captions in Text

In [29]:
with open("list-bliplarge.txt", "w") as file:
    for item in combined_docs:
        file.write(str(item) + "\n")

## if we don't like to save captions in Text we can directly Convert text to LAngchain documents

In [34]:
# Load the dynamic text into a LangChain document
docs = [Document(page_content=text) for text in combined_docs ]


## load Texts

In [5]:

loader = TextLoader('/Users/bpakra200/tubegpt/tubegpt/src/notebook/list.txt')

docs = loader.load()

In [6]:
len(docs)

1

In [30]:
loader = TextLoader('/Users/bpakra200/tubegpt/tubegpt/src/notebook/list-bliplarge.txt')

docs = loader.load()

## Define embedding

In [31]:
embeddings = OpenAIEmbeddings()

## Save Vectors in Database base model

In [11]:
PERSIST_DIRECTORY_OPENAI = "../tubegptvisiondb/"
 
tubegptvisiondb = Chroma.from_documents(
    documents=docs, embedding= embeddings, persist_directory=PERSIST_DIRECTORY_OPENAI
)
tubegptvisiondb.similarity_search_with_score("What was weman were playing?", k=2)

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


[(Document(page_content='a man in a green golf cart with a stuffed bear\na golf cart with a christmas sweater on it\na black bear is sitting in a golf cart\na man is playing with a frc in the yard\na green truck\na christmas wreath with a stuffed bear in it\ntwo boys playing golf in the grass\na group of people playing golf on a green field\na santa claus wreath is placed on a tractor\ntwo children playing with a fr fr fr fr fr fr fr fr fr fr fr fr fr fr\na man driving a golf cart with a christmas hat on\na stuffed santa claus is sitting in a golf cart\na santa claus in a green truck\na stuffed animal sitting in the back of a truck\na christmas wreath with a santa claus in it\na stuffed santa claus is sitting in a truck\na dog in a christmas wreath sitting on a truck\na stuffed bear sitting in a christmas wreath\na green golf cart with a santa hat on it\na man and a woman playing with a frc\na stuffed santa claus is placed in the back of a truck\na green golf cart with a wreath on it\n

## Save vecrtor for large captioning models

In [32]:
PERSIST_DIRECTORY_OPENAI = "../tubegptvisionlargedb/"
 
tubegptvisionlargedb = Chroma.from_documents(
    documents=docs, embedding= embeddings, persist_directory=PERSIST_DIRECTORY_OPENAI
)
tubegptvisionlargedb.similarity_search_with_score("What was weman were playing?", k=2)

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


[(Document(page_content='there is a stuffed santa claus doll on a wreath on the back of a truck\nthere is a stuffed santa claus doll on a wreath on the back of a car\nthere is a stuffed santa claus on a wreath on the back of a car\nthere is a stuffed santa claus sitting on a wreath on the back of a car\nthere is a stuffed santa claus sitting on a wreath on the back of a golf cart\nthere is a stuffed santa claus on a wreath on the back of a golf cart\nthere is a stuffed santa claus sitting on a wreath on the back of a truck\nthere is a stuffed animal that is sitting on a wreath\nthere is a wreath with a santa clause on it on the back of a car\nthere is a stuffed animal in a wreath on the back of a car\nthere is a stuffed santa claus on a wreath on the back of a truck\nthere is a stuffed animal in a wreath on the back of a truck\nthere is a stuffed santa claus sitting on a christmas wreath\nthere is a stuffed santa claus doll sitting on a christmas wreath\nthere is a stuffed animal that 

In [12]:

tubegptvisiondb.persist()


In [33]:
tubegptvisionlargedb.persist()

## load Vectors from database

In [34]:
PERSIST_DIRECTORY_OPENAI =  "../tubegptvisionlargedb"
embeddings = OpenAIEmbeddings()
tubegptvisionlargedb = Chroma(persist_directory=PERSIST_DIRECTORY_OPENAI, embedding_function=embeddings)
tubegptvisionlargedb.similarity_search_with_score("are they playing golf", k=2)

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


[(Document(page_content='there is a stuffed santa claus doll on a wreath on the back of a truck\nthere is a stuffed santa claus doll on a wreath on the back of a car\nthere is a stuffed santa claus on a wreath on the back of a car\nthere is a stuffed santa claus sitting on a wreath on the back of a car\nthere is a stuffed santa claus sitting on a wreath on the back of a golf cart\nthere is a stuffed santa claus on a wreath on the back of a golf cart\nthere is a stuffed santa claus sitting on a wreath on the back of a truck\nthere is a stuffed animal that is sitting on a wreath\nthere is a wreath with a santa clause on it on the back of a car\nthere is a stuffed animal in a wreath on the back of a car\nthere is a stuffed santa claus on a wreath on the back of a truck\nthere is a stuffed animal in a wreath on the back of a truck\nthere is a stuffed santa claus sitting on a christmas wreath\nthere is a stuffed santa claus doll sitting on a christmas wreath\nthere is a stuffed animal that 

In [ ]:
PERSIST_DIRECTORY_OPENAI =  "../tubegptvisiondb/"
embeddings = OpenAIEmbeddings()
tubegptvisiondb = Chroma(persist_directory=PERSIST_DIRECTORY_OPENAI, embedding_function=embeddings)
tubegptvisiondb.similarity_search_with_score("are they playing golf", k=2)

## Create a prompt Templete

In [15]:
template = """You're a Helper of poor vision person who needs help to understand a video content .
 
{context}
 
Answer with Accurate data   to the question and the way Helper of  poor vision person speaks and only depends on provided Data. 
 
Question: {question}
Answer:"""
 
prompt = PromptTemplate(template=template, input_variables=["context", "question"])
print(
    prompt.format(
        context="A youtube user asking about contents ",
        question="Why do we need to zero out the gradient before backprop at each step?",
    )
)

You're a Helper of poor vision person who needs help to understand a video content .
 
A youtube user asking about contents 
 
Answer with Accurate data   to the question and the way Helper of  poor vision person speaks and only depends on provided Data. 
 
Question: Why do we need to zero out the gradient before backprop at each step?
Answer:


## Create Retriver and Reader from caption Data for base model

In [18]:
chain_type_kwargs = {"prompt": prompt}
chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=.4),
    chain_type="stuff",
    retriever=tubegptvisiondb.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
)

## Create Retriver and Reader from caption Data for large captioning model

In [35]:
chain_type_kwargs = {"prompt": prompt}
chainlarge = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=.4),
    chain_type="stuff",
    retriever=tubegptvisionlargedb.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
)

## Asking questions on Video

In [102]:
query = "What was game they are playing"
response = chain.run(query)
print_response(response)

The game they are playing is golf.


In [103]:
query = "What was game they are playing"
response = chainlarge.run(query)
print_response(response)

The game they are playing is golf.


In [22]:
query = "Why did the two ladies put their hands above their eyes while staring out?"
response = chain.run(query)
print_response(response)

Based on the provided data, there is no information about two ladies putting their hands above their
eyes while staring out.


In [115]:
query = "Why did the two ladies put their hands above their eyes while staring out?"
response = chainlarge.run(query)
print_response(response)

I'm sorry, but based on the provided data, there is no information about two ladies putting their
hands above their eyes while staring out.


In [104]:
query = "what was the color of golf cart?"
response = chainlarge.run(query)
print_response(response)

The color of the golf cart is green.


In [105]:
query = "what was the color of golf cart?"
response = chain.run(query)
print_response(response)

The color of the golf cart was green.


In [40]:
query = "what the color of the pant people wearing?"
response = chain.run(query)
print_response(response)

I apologize, but the provided data does not mention anything about the color of the pants people are
wearing.


In [41]:
query = "what the color of the pant people wearing?"
response = chainlarge.run(query)
print_response(response)

I'm sorry, but the provided data does not mention anything about the color of the pants people are
wearing.


In [42]:
query = "how many people playing?"
response = chainlarge.run(query)
print_response(response)

There are several people playing in the given video content.


In [110]:
query = "what is the number on golf cart?"
response = chainlarge.run(query)
print_response(response)

The provided data does not include any information about the number on the golf cart.


In [107]:
query = "Can you let me know if it was a sunny day?"
response = chainlarge.run(query)
print_response(response)

Based on the provided data, there is no information about the weather or whether it was a sunny day.


In [109]:
query = "Can you describe the golf cart?"
response = chainlarge.run(query)
print_response(response)

The golf cart is green in color and has a wreath on the back. It may also have a stuffed animal or a
Christmas decoration on it.


In [111]:
query = "Can you describe the background?"
response = chainlarge.run(query)
print_response(response)

Based on the provided data, the background appears to be a park or a grassy field. People are
playing various activities such as golf or frisbee. There are also golf carts present in the
background. The weather seems to be sunny.


In [114]:
query = "Is it sunny during the game ?"
response = chainlarge.run(query)
print_response(response)

Based on the provided data, there is no information about the weather conditions during the game.
